In [1]:
#Superimpose and measure the code between two TMDs of PDBs from two GPCRs PDBs

In [2]:
from Bio.PDB import PDBParser, Superimposer, PDBIO
import numpy as np

def get_alpha_carbons(residues):
    alpha_carbons = []
    for residue in residues:
        alpha_carbon = residue['CA']
        if alpha_carbon is not None:
            alpha_carbons.append(alpha_carbon)
    return alpha_carbons

def get_residues_by_range(chain, start_residue, end_residue):
    residues = []
    for residue in chain:
        if start_residue <= residue.id[1] <= end_residue:
            residues.append(residue)
    return residues

def calc_rotation_angle(rotation_matrix):
    angle_of_rotation = np.arccos((np.trace(rotation_matrix) - 1) / 2.0) * (180.0 / np.pi)
    return angle_of_rotation

def calc_rmsd(atom_list1, atom_list2):
    diff = np.array([atom_list1[i].get_coord() - atom_list2[i].get_coord() for i in range(len(atom_list1))])
    rmsd = np.sqrt(np.sum(diff**2) / len(atom_list1))
    return rmsd

def align_and_superimpose(pdb_file1, pdb_file2, chain_id, helix_ranges, output_pdb_file):
    # Load the two structures
    parser = PDBParser(QUIET=True)
    structure1 = parser.get_structure('structure1', pdb_file1)
    structure2 = parser.get_structure('structure2', pdb_file2)

    # Get the specified chains
    chain_A1 = structure1[0][chain_id]
    chain_A2 = structure2[0][chain_id]

    fixed_residues = get_residues_by_range(chain_A1, *helix_ranges[0])
    moving_residues = get_residues_by_range(chain_A2, *helix_ranges[0])

    fixed_atoms = get_alpha_carbons(fixed_residues)
    moving_atoms = get_alpha_carbons(moving_residues)

    # Check if the number of alpha carbons is the same for both structures
    if len(fixed_atoms) != len(moving_atoms):
        raise ValueError("Error: Fixed and moving atom lists differ in size.")

    # Perform superimposition
    superimposer = Superimposer()
    superimposer.set_atoms(fixed_atoms, moving_atoms)

    # Apply the transformation to the moving structure to align it with the fixed structure
    superimposer.apply(structure2[0].get_atoms())

    # Calculate the rotation matrix for the final superimposition
    rotation_matrix = superimposer.rotran[0]

    # Save the superimposed structure to a new PDB file
    io = PDBIO()
    io.set_structure(structure2)
    io.save(output_pdb_file)

    # Calculate the RMSD to determine the average rotation
    rmsd = calc_rmsd(fixed_atoms, moving_atoms)

    # Calculate the angle of rotation
    angle_of_rotation = calc_rotation_angle(rotation_matrix)

    return rmsd, angle_of_rotation

In [3]:
pdb_file1 = "/Users/ayodi/OneDrive/Documents/lpa_data/newlpa_data/inactive_lpa.pdb"
pdb_file2 = "/Users/ayodi/OneDrive/Documents/lpa_data/newlpa_data/activenew.pdb"
chain_id = "A"  # Replace 'A' with the chain ID of interest

In [7]:
# Specify the helix ranges as (start_residue, end_residue)
helix_ranges = [(251, 265), (251, 265)]  # Add more helix ranges if needed
output_pdb_file = "/Users/ayodi/OneDrive/Documents/lpa_data/newlpa_data/output_superimposed.pdb"
try:
    rmsd, rotation_angle = align_and_superimpose(pdb_file1, pdb_file2, chain_id, helix_ranges, output_pdb_file)
    print(f"Average RMSD between the two helices: {rmsd:.2f} angstroms")
    print(f"Rotation Angle between the two helices: {rotation_angle:.2f} degrees")
except Exception as e:
    print("An error occurred:", str(e))

Average RMSD between the two helices: 0.94 angstroms
Rotation Angle between the two helices: 44.28 degrees
